In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

In [2]:
# URL of page to be scraped
url = 'http://www.espn.com/nhl/statistics/player/_/stat/points/sort/points/year/2020/seasontype/2'

In [3]:
# Retrieve page with the requests module
response = requests.get(url)
print(response.status_code)

200


In [4]:
html=response.content

In [5]:
soup =bs(html,"lxml")

In [6]:
headers = soup.find_all('tr', attrs= {'class':"colhead"})
header =headers[1]
columns = [col.get_text() for col in header.find_all("td")]
df = pd.DataFrame(columns =columns)
df

,RK,PLAYER,TEAM,GP,G,A,PTS,+/-,PIM,PTS/G,SOG,PCT,GWG,G,A,G,A


In [7]:
for i in range(1,798,40):
    url = "http://www.espn.com/nhl/statistics/player/_/stat/points/sort/points/count/{}".format(i)
    page = requests.get(url)
    soup = bs(page.text, 'lxml')
    
    players = soup.find_all("tr", attrs={'class':re.compile('row player-90-')})
    for player in players:
        stats = [stat.get_text() for stat in player.find_all('td')]
        
        temp_df = pd.DataFrame(stats).transpose()
        temp_df.columns = columns
        
        df = pd.concat([df, temp_df],ignore_index = True)
    
df
    

,RK,PLAYER,TEAM,GP,G,A,PTS,+/-,PIM,PTS/G,SOG,PCT,GWG,G,A,G,A
0,1,"Leon Draisaitl, C",EDM,71,43,67,110,-7,18,1.55,218,19.7,10,16,28,0,0
1,2,"Connor McDavid, C",EDM,64,34,63,97,-6,28,1.52,212,16.0,6,11,32,0,0
2,3,"David Pastrnak, RW",BOS,70,48,47,95,21,40,1.36,279,17.2,10,20,18,0,0
3,,"Artemi Panarin, LW",NYR,69,32,63,95,36,20,1.38,209,15.3,4,7,17,0,0
4,5,"Nathan MacKinnon, C",COL,69,35,58,93,13,12,1.35,318,11.0,4,12,19,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,,"Paul LaDue, D",LA,2,0,1,1,0,0,0.50,2,.0,0,0,0,0,0
794,,"Pontus Aberg, LW",TOR,5,0,1,1,0,0,0.20,4,.0,0,0,0,0,0
795,,"Ryan MacInnis, C",CBJ,10,0,1,1,1,0,0.10,13,.0,0,0,0,0,0
796,,"Clark Bishop, C",CAR,5,0,1,1,-2,4,0.20,4,.0,0,0,0,0,0


In [8]:
df_clean = df.iloc[:,[1,2,3,4,5,6,8]]
df_clean

,PLAYER,TEAM,GP,G,A,PTS,PIM
0,"Leon Draisaitl, C",EDM,71,43,67,110,18
1,"Connor McDavid, C",EDM,64,34,63,97,28
2,"David Pastrnak, RW",BOS,70,48,47,95,40
3,"Artemi Panarin, LW",NYR,69,32,63,95,20
4,"Nathan MacKinnon, C",COL,69,35,58,93,12
...,...,...,...,...,...,...,...
793,"Paul LaDue, D",LA,2,0,1,1,0
794,"Pontus Aberg, LW",TOR,5,0,1,1,0
795,"Ryan MacInnis, C",CBJ,10,0,1,1,0
796,"Clark Bishop, C",CAR,5,0,1,1,4


In [9]:
new = df_clean["PLAYER"].str.split(" ", n = 1, expand = True)

In [10]:
ps = new[1].str.split(",",n=1, expand =True)

In [11]:
df_clean["first_name"] = new[0]
df_clean["last_name"]=ps[0]
df_clean["position"]=ps[1]

<ipython-input-11-ad1a590529a4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["first_name"] = new[0]
<ipython-input-11-ad1a590529a4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["last_name"]=ps[0]
<ipython-input-11-ad1a590529a4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [12]:
df_clean

,PLAYER,TEAM,GP,G,A,PTS,PIM,first_name,last_name,position
0,"Leon Draisaitl, C",EDM,71,43,67,110,18,Leon,Draisaitl,C
1,"Connor McDavid, C",EDM,64,34,63,97,28,Connor,McDavid,C
2,"David Pastrnak, RW",BOS,70,48,47,95,40,David,Pastrnak,RW
3,"Artemi Panarin, LW",NYR,69,32,63,95,20,Artemi,Panarin,LW
4,"Nathan MacKinnon, C",COL,69,35,58,93,12,Nathan,MacKinnon,C
...,...,...,...,...,...,...,...,...,...,...
793,"Paul LaDue, D",LA,2,0,1,1,0,Paul,LaDue,D
794,"Pontus Aberg, LW",TOR,5,0,1,1,0,Pontus,Aberg,LW
795,"Ryan MacInnis, C",CBJ,10,0,1,1,0,Ryan,MacInnis,C
796,"Clark Bishop, C",CAR,5,0,1,1,4,Clark,Bishop,C


In [13]:
df_final = df_clean.iloc[:,[1,2,3,4,5,6,7,8,9]]
df_final

,TEAM,GP,G,A,PTS,PIM,first_name,last_name,position
0,EDM,71,43,67,110,18,Leon,Draisaitl,C
1,EDM,64,34,63,97,28,Connor,McDavid,C
2,BOS,70,48,47,95,40,David,Pastrnak,RW
3,NYR,69,32,63,95,20,Artemi,Panarin,LW
4,COL,69,35,58,93,12,Nathan,MacKinnon,C
...,...,...,...,...,...,...,...,...,...
793,LA,2,0,1,1,0,Paul,LaDue,D
794,TOR,5,0,1,1,0,Pontus,Aberg,LW
795,CBJ,10,0,1,1,0,Ryan,MacInnis,C
796,CAR,5,0,1,1,4,Clark,Bishop,C


In [14]:
df_final = df_final.rename(columns ={"TEAM":"team"})

In [15]:
df_final =df_final[["first_name", "last_name", "position", "team", "GP", "G","A","PTS","PIM"]]
df_final

,first_name,last_name,position,team,GP,G,A,PTS,PIM
0,Leon,Draisaitl,C,EDM,71,43,67,110,18
1,Connor,McDavid,C,EDM,64,34,63,97,28
2,David,Pastrnak,RW,BOS,70,48,47,95,40
3,Artemi,Panarin,LW,NYR,69,32,63,95,20
4,Nathan,MacKinnon,C,COL,69,35,58,93,12
...,...,...,...,...,...,...,...,...,...
793,Paul,LaDue,D,LA,2,0,1,1,0
794,Pontus,Aberg,LW,TOR,5,0,1,1,0
795,Ryan,MacInnis,C,CBJ,10,0,1,1,0
796,Clark,Bishop,C,CAR,5,0,1,1,4


In [23]:
df_players = df_final[["first_name","last_name", "team", "position"]]
df_player_stats = df_final[["first_name", "last_name", "GP", "G", "A", "PTS", "PIM"]]

In [17]:
df_players

,first_name,last_name,team,position
0,Leon,Draisaitl,EDM,C
1,Connor,McDavid,EDM,C
2,David,Pastrnak,BOS,RW
3,Artemi,Panarin,NYR,LW
4,Nathan,MacKinnon,COL,C
...,...,...,...,...
793,Paul,LaDue,LA,D
794,Pontus,Aberg,TOR,LW
795,Ryan,MacInnis,CBJ,C
796,Clark,Bishop,CAR,C


In [18]:
df_player_stats

,first_name,last_name,GP,G,A,PTS,PIM
0,Leon,Draisaitl,71,43,67,110,18
1,Connor,McDavid,64,34,63,97,28
2,David,Pastrnak,70,48,47,95,40
3,Artemi,Panarin,69,32,63,95,20
4,Nathan,MacKinnon,69,35,58,93,12
...,...,...,...,...,...,...,...
793,Paul,LaDue,2,0,1,1,0
794,Pontus,Aberg,5,0,1,1,0
795,Ryan,MacInnis,10,0,1,1,0
796,Clark,Bishop,5,0,1,1,4


In [21]:
df_final.to_csv(r'df_final.csv')
df_players.to_csv(r'players.csv')
df_player_stats.to_csv(r'player_stats.csv')